In [315]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 
import operator
import random
import math


In [316]:
def houghTransform(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((15, 15), np.uint8)

    opening = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)

    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    cv2.imwrite("edges.jpg", edges)

    # Compute the Hough Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)
    print("Found lines: {}".format(len(lines)))
    houghLines = []
    # Draw the lines
    for line in lines:
        rho, theta = line[0]
        if abs(theta-np.pi/90) < np.pi/9:
            continue
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        line_points = list(((x1,y1), (x2,y2)))
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
        cv2.imwrite("StandardHough.jpg", img);
        # print("line_points: ", line_points)
        houghLines.append( line_points)
        # cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        fhLines = filteringHoughLines(edges, img, houghLines)
    return fhLines

In [318]:
def randomSampleLines(lines):
    """
    Randomly sample lines from the given lines.
    :param lines: The lines to sample from.
    :return: The sampled lines.
    """
    # print("lines",lines)
    # print(len(lines))
    num = 100
    if num > len(lines):
        num = len(lines)
    # print("num",num)
    x = random.sample(lines, min(len(lines), num))
    return x

In [319]:
def determine(a, b):
    return a[0] * b[1] - a[1] * b[0]
def intersectionOftwoLines(line_1,line_2):
    '''
    finds the intersection point between two lines
    '''
    '''
    convert tuple to list
    '''
    xVal = (line_1[0][0] - line_1[1][0], line_2[0][0] - line_2[1][0])
    yVal = (line_1[0][1] - line_1[1][1], line_2[0][1] - line_2[1][1])

    intersect = determine(xVal, yVal)

    if intersect == 0:
        return 0,0

    temp = (determine(*line_1), determine(*line_2))
    x = determine(temp, xVal) / intersect
    y = determine(temp, yVal) / intersect

    return x, y

def intersectionofmultipleLines(lines):
    '''
    finds intersection between multiple lines
    '''
    intersections = []
    for i in range(len(lines)):
        for j in range(i+1, len(lines)):
            # print(lines[i])
            # print(lines[j])
            intersection = intersectionOftwoLines(lines[i], lines[j])
            if intersection not in intersections:
                intersections.append(intersection)
    return intersections

In [320]:
def distancebetweenTwoPoints(p1, p2):
    '''
    calculate distance between point tuple
    '''
    x1, y1 = p1
    x2, y2 = p2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def checklineIntersection(line, p1, p2):
    '''
    check if a line passes throguh two points
    '''
    length1 = distancebetweenTwoPoints((line[0][0],line[0][1]),(p1, p2))    
    length2 = distancebetweenTwoPoints((line[1][0],line[1][1]),(p1, p2))
    length3 = distancebetweenTwoPoints((line[0][0],line[0][1]),(line[1][0],line[1][1]))

    if int(length1) + int(length2)  == int(length3):
        return True
    else:
        return False


In [321]:
def twoLineVP(img, houghLines):
    '''
    function to get the Vanishing point using two lines
    '''
    intersectionCounter = {}
    
    for line1 in houghLines:
        for line2 in houghLines:
            # if both the lines are same then don't process
            if line1 == line2:
                continue
            
            # check if point the lines are intersecting
            # print(line1, line2)
            # print(intersectionOftwoLines(line1, line2))
            pointX, pointY = intersectionOftwoLines(line1, line2)
            intersectionCounter[(pointX, pointY)] = 0

            # now check for all other lines that intersect with this pair
            for lines in houghLines:
                # if lines are same then don't process
                if lines == line1 or lines == line2:
                    continue

                # check if the line is passing through the pair of points
                flag =  checklineIntersection(lines, pointX, pointY)
                if flag:
                    intersectionCounter[(pointX, pointY)] += 1

    finalIntersectionCounter = max(intersectionCounter.items(), key=operator.itemgetter(1))[0]
    print(finalIntersectionCounter)
    print(type (finalIntersectionCounter))
    print(finalIntersectionCounter[0])
    print(finalIntersectionCounter[1])
    cv2.circle(image, (int(finalIntersectionCounter[0]), int(finalIntersectionCounter[1])), 55, (0, 255, 0), 2)
    cv2.imwrite("LinePairVP.jpg", image)
    print (finalIntersectionCounter)



In [322]:
import itertools
# Given intersections, find the grid where most intersections occur and treat as vanishing point
def vanishingPointCalculation(img, grid_size, intersections):
    # Image dimensions
    image_height = img.shape[0]
    image_width = img.shape[1]

    # Grid dimensions
    grid_rows = (image_height // grid_size) + 1
    grid_columns = (image_width // grid_size) + 1

    # Current cell with most intersection points
    max_intersections = 0
    best_cell = (0.0, 0.0)

    for i, j in itertools.product(range(grid_columns),range(grid_rows)):
        cell_left = i * grid_size
        cell_right = (i + 1) * grid_size
        cell_bottom = j * grid_size
        cell_top = (j + 1) * grid_size
        cv2.rectangle(img, (cell_left, cell_bottom), (cell_right, cell_top), (0, 0, 255), 10)

        current_intersections = 0  # Number of intersections in the current cell
        for x, y in intersections:
            if cell_left < x < cell_right and cell_bottom < y < cell_top:
                current_intersections += 1

        # Current cell has more intersections that previous cell (better)
        if current_intersections > max_intersections:
            max_intersections = current_intersections
            best_cell = ((cell_left + cell_right) / 2, (cell_bottom + cell_top) / 2)
            print("Best Cell:", best_cell)

    if best_cell[0] != None and best_cell[1] != None:
        rx1 = int(best_cell[0] - grid_size / 2)
        ry1 = int(best_cell[1] - grid_size / 2)
        rx2 = int(best_cell[0] + grid_size / 2)
        ry2 = int(best_cell[1] + grid_size / 2)
        cv2.rectangle(img, (rx1, ry1), (rx2, ry2), (0, 255, 0), 10)
        cv2.imwrite('/pictures/output/center.jpg', img)

    return best_cell

## Method 1

In [323]:
image = cv2.imread("/home/appledora/Documents/gig/vanishing-point/Corridor.jpg")
houghLines = houghTransform(image)
twoLineVP(image, fhLines)


Found lines: 37


NameError: name 'bresenham_line' is not defined

## Method 2

In [293]:

if houghLines:
        random_sample = randomSampleLines(houghLines)
        intersections = intersectionofmultipleLines(random_sample)
        if intersections:
            grid_size = min(image.shape[0], image.shape[1]) // 5
            vanishing_point = vanishingPointCalculation(image, grid_size, intersections)
            cv2.circle(image, (int(vanishing_point[0]), int(vanishing_point[1])), 2, (0, 255, 0), 2)
            cv2.imwrite("otherone.jpg", image)
            

Best Cell: (540.0, 108.0)
